In [72]:
import pandas as pd
import numpy as np

import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import pairwise_distances

import spacy

from tqdm import tqdm
tqdm.pandas()

from nltk.sentiment.vader import SentimentIntensityAnalyzer

import itertools

from rouge_score import rouge_scorer
from bert_score import score
from nltk.translate import bleu_score

import statistics

# Data Loading 

In [73]:
DATA_SOURCE = '../../data/data_clean/polisumm_final.csv'

In [74]:
data = pd.read_csv(DATA_SOURCE)

# Filter Out Adjectives 

In [75]:
nlp = spacy.load('en_core_web_sm')
url_regex = r'https?:\/\/\S*'

In [76]:
def remove_sal_terms(text):
    doc = nlp(text)
    new_doc = []
    for token in doc:
        if token.pos_ != 'ADJ' and token.dep_ != 'neg':
            new_doc.append(token.text)
    
    return ' '.join(new_doc)

def process_text(text):
    text = re.sub(url_regex, '', str(text))
    text_l = re.split(punct_regex, text)
    text_l = [t for t in text_l if t not in ('', ' ')]
    
    new_text_l = []
    
    for single_text in text_l:
        new_text_l.append(remove_sal_terms(single_text))
    
    return ' | '.join(new_text_l)

In [285]:
data['tweet_text_proc'] = data['tweet_text'].progress_apply(process_text)
data['reddit_text_proc'] = data['reddit_text'].progress_apply(process_text)

100%|██████████████████████████████████████████████████████████████████████████████| 1199/1199 [05:59<00:00,  3.34it/s]


In [286]:
data.to_csv('../../data/data_clean/polisumm_final_cmos.csv', index = None)

# Sentiment for Sorting

In [287]:
punct_regex = '\s*[\.\?\!\|\n\:]\s*[\.\?\!\|\n\:]*\s*'

In [288]:
sid = SentimentIntensityAnalyzer()

In [289]:
def get_sentiments(text):
    text = re.sub(url_regex, '', str(text))
    text_l = re.split(punct_regex, text)
    text_l = [t for t in text_l if t != '']
    
    sent_scores = []
    for sent in text_l:
        sentiments = sid.polarity_scores(sent)
        sent_scores.append(sentiments)
    
    return sent_scores

In [290]:
data['tweet_sent_scores'] = data['tweet_text'].progress_apply(get_sentiments)
data['reddit_sent_scores'] = data['reddit_text'].progress_apply(get_sentiments)

100%|█████████████████████████████████████████████████████████████████████████████| 1199/1199 [00:08<00:00, 144.26it/s]


In [291]:
data.to_csv('../../data/data_clean/polisumm_final_cmos.csv', index = None)

In [77]:
data = pd.read_csv('../../data/data_clean/polisumm_final_cmos.csv')

# Combine 

In [78]:
def combine_sents(row):
    sent_tw = row['tweet_sent_scores']
    sent_rd = row['reddit_sent_scores']
    return (sent_tw + sent_rd)

In [79]:
data['all_texts'] = data['tweet_text'] + ' ' + data['reddit_text']
data['all_texts_proc'] = data['tweet_text_proc'].fillna('') + ' ' + data['reddit_text_proc'].fillna('')
data['all_sent_scores'] = data.progress_apply(combine_sents, axis = 1)

100%|███████████████████████████████████████████████████████████████████████████| 1199/1199 [00:00<00:00, 15883.65it/s]


# Vectorizer Training 

In [80]:
vect = CountVectorizer(min_df = 5, max_df = 0.8)
tf_vect = CountVectorizer(min_df = 5, max_df = 0.8)

In [81]:
vect = vect.fit(data['all_texts_proc'])
tf_vect = tf_vect.fit(data['all_texts_proc'])

# CMOS Model 

In [82]:
class CMOS():
    
    def __init__(self, lambda_w, vectorizer):
        self.lambda_w = lambda_w
        self.vectorizer = vectorizer
        self.PUNCT_REGEX = '\s*[\.\?\!\|\n\:]\s*[\.\?\!\|\n\:]*\s*'
    
    def predict(self, text, sentiments):
        '''
            Text should not contain adjectives or negations
        '''
        
        text_l = self.split_text(text) if isinstance(text, str) else text
        
        sent_scores = [sent['compound'] for sent in sentiments]
        
        min_len = min(len(text_l), len(sent_scores))
        text_l, sent_scores = text_l[:min_len], sent_scores[:min_len]
        assert len(text_l) == len(sent_scores), 'Text and sentiment scores are of different lengths'
        
        text_sents = [(stext, ssent) for stext, ssent in zip(text_l, sent_scores)]
        text_sents.sort(key = lambda pair: pair[1])
        
        half1 = [t[0] for t in text_sents[:len(text_sents)//2]]
        half2 = [t[0] for t in text_sents[len(text_sents)//2:]]
        
        half1_bows = self.get_vecs(half1)
        half2_bows = self.get_vecs(half2)
        
        half1_cs = self.calc_cos_sims(half1_bows)
        half2_cs = self.calc_cos_sims(half2_bows)
        sum_cs   = self.combine_cos_sims(half1_cs, half2_cs)
        
        dist_cs  = self.calc_cos_dists(half1_bows, half2_bows)
        
        all_scores    = sum_cs + 2 * self.lambda_w * dist_cs
        h1_idx, h2_idx = np.unravel_index(all_scores.argmax(), all_scores.shape)
        
        return half1[h1_idx], half2[h2_idx]
    
    def calc_cos_sims(self, bows):
        sims = 1. - pairwise_distances(bows, metric = 'cosine')
        sims = sims.mean(-1)
        return sims
    
    def combine_cos_sims(self, cs1s, cs2s):
        combos = itertools.product(cs1s, cs2s)
        combos = [a + b for (a, b) in combos]
        combos = np.array(combos).reshape(len(cs1s), len(cs2s))
        return combos
    
    def calc_cos_dists(self, bows1, bows2):
        dists = pairwise_distances(bows1, bows2, metric = 'cosine')
        return dists
    
    def get_vecs(self, text_list):
        return self.vectorizer.transform(text_list)
    
    def split_text(self, text):
        text_l = re.split(self.PUNCT_REGEX, text)
        text_l = [t for t in text_l if t != '']
        return text_l

# Evaluation 

In [7]:
refs = data['left_sum'] + ' | ' + data['right_sum']

In [304]:
cmos    = CMOS(0.5, vect)
tf_cmos = CMOS(0.5, tf_vect)

In [306]:
def make_prediction(model, row):
    text = row['all_texts_proc']
    sent_scores = row['all_sent_scores']
    
    summ_1, summ_2 = model.predict(text, sent_scores)
    return summ_1 + ' | ' + summ_2

In [307]:
predictions = data.progress_apply(lambda row: make_prediction(cmos, row), axis = 1)

100%|██████████████████████████████████████████████████████████████████████████████| 1199/1199 [01:43<00:00, 11.58it/s]


In [83]:
# predictions.to_csv('preds_cmos.csv')
predictions = pd.read_csv('preds_cmos.csv')

In [87]:
predictions['pred1'] = predictions['0'].str.split('|').apply(lambda l: l[0])
predictions['pred2'] = predictions['0'].str.split('|').apply(lambda l: l[1])

In [95]:
eval_split = pd.concat((predictions[['pred1', 'pred2', '0']], data[['left_sum', 'right_sum']]), axis = 1)

In [96]:
eval_split['ref_comb'] = eval_split['left_sum'] + '|' + eval_split['right_sum']
eval_split['ref_comb_rev'] = eval_split['right_sum'] + '|' + eval_split['left_sum']

In [99]:
eval_split = eval_split.dropna()

### BERT Score 

In [32]:
bert_scores = []
for i, (pred, ref) in enumerate(zip(eval_set['preds'].astype(str).values, eval_set['refs'].astype(str).values)):

    bert_score = score([pred], [ref], 
                       lang = 'en',
                      use_fast_tokenizer = True,
                      batch_size = 1,
                       model_type = 'bert-base-uncased'
                      )
    
    bert_scores.append(bert_score)
    print(f'Completed {i + 1} of {len(eval_set)}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 0 of 898


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 1 of 898


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Completed 895 of 898


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 896 of 898


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Completed 897 of 898


In [35]:
bert_scores_proc = {'prec': [], 'rec': [], 'f1': []}
for bscore in bert_scores:
    bert_scores_proc['prec'].append(bscore[0].item())
    bert_scores_proc['rec'].append(bscore[1].item())
    bert_scores_proc['f1'].append(bscore[2].item())

In [36]:
with open('bert_scores.txt', 'w') as f:
    f.write(str(bert_scores_proc))

In [37]:
bprec = sum(bert_scores_proc['prec'])/len(bert_scores_proc['prec'])
brec  = sum(bert_scores_proc['rec']) / len(bert_scores_proc['rec'])
bf1   = sum(bert_scores_proc['f1']) / len(bert_scores_proc['f1'])

In [143]:
bf1

0.4519044154869156

### Diversity (Self-Bleu) 

In [46]:
split_predictions = [pred.split('|') for pred in predictions['0']]

In [47]:
def get_self_bleu(split_preds):
    self_bleus = []
    for pred_pair in split_preds:
        bleu   = bleu_score.sentence_bleu([pred_pair[0]], pred_pair[1], weights = (1.0,))
        self_bleus.append(bleu)
    return self_bleus

In [48]:
self_bleu = get_self_bleu(split_predictions)

### ROUGE Scores 

In [91]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer = True)

In [133]:
def get_rouge_scores(df, src_col, ref_col):
    rouge_scores = []
    for i, (pred, ref) in enumerate(zip(df[src_col].astype(str).values, df[ref_col].astype(str).values)):
        rscore = scorer.score(ref, pred)
        rouge_scores.append(rscore)
        print(f'Done with {i} of {len(df)}', end = '\r')
    return rouge_scores

In [135]:
pred1_l_rouge = get_rouge_scores(eval_split, 'pred1', 'left_sum')
pred1_r_rouge = get_rouge_scores(eval_split, 'pred1', 'right_sum')
pred2_l_rouge = get_rouge_scores(eval_split, 'pred2', 'left_sum')
pred2_r_rouge = get_rouge_scores(eval_split, 'pred2', 'right_sum')
comb_rouge    = get_rouge_scores(eval_split, '0', 'ref_comb')
comb_rouge_r  = get_rouge_scores(eval_split, '0', 'ref_comb_rev')

In [136]:
all_results = [pred1_l_rouge, pred1_r_rouge, pred2_l_rouge, pred2_r_rouge, comb_rouge, comb_rouge_r]
all_results_names = ['1l', '1r', '2l', '2r', 'c', 'cr']

In [137]:
all_results = [pd.DataFrame(res) for res in all_results]
for res in all_results:
    for column in res.columns:
        res[column] = res[column].apply(lambda scores: scores[-1])

In [138]:
for name, res in zip(all_results_names, all_results):
    res.columns = [name + '_' + col for col in res.columns]

In [139]:
result_data = pd.concat(all_results, axis = 1)

In [140]:
result_data['ml_rouge1'] = result_data[['1l_rouge1', '2l_rouge1']].max(axis = 1)
result_data['ml_rouge2'] = result_data[['1l_rouge2', '2l_rouge2']].max(axis = 1)
result_data['ml_rougeL'] = result_data[['1l_rougeL', '2l_rougeL']].max(axis = 1)
result_data['mr_rouge1'] = result_data[['1r_rouge1', '2r_rouge1']].max(axis = 1)
result_data['mr_rouge2'] = result_data[['1r_rouge2', '2r_rouge2']].max(axis = 1)
result_data['mr_rougeL'] = result_data[['1r_rougeL', '2r_rougeL']].max(axis = 1)
result_data['mc_rouge1'] = result_data[['c_rouge1', 'cr_rouge1']].max(axis = 1)
result_data['mc_rouge2'] = result_data[['c_rouge2', 'cr_rouge2']].max(axis = 1)
result_data['mc_rougeL'] = result_data[['c_rougeL', 'cr_rougeL']].max(axis = 1)

In [141]:
result_data[['ml_rouge1', 'ml_rouge2', 'ml_rougeL', 'mr_rouge1', 'mr_rouge2', 'mr_rougeL', 'mc_rouge1', 'mc_rouge2', 'mc_rougeL']].mean()

ml_rouge1    0.160013
ml_rouge2    0.018021
ml_rougeL    0.133452
mr_rouge1    0.157684
mr_rouge2    0.016231
mr_rougeL    0.134386
mc_rouge1    0.144144
mc_rouge2    0.014478
mc_rougeL    0.120324
dtype: float64

In [142]:
result_data.to_csv('results.csv', index = None)

In [51]:
rouge1 = {'prec': [], 'rec': [], 'f1': []}
rouge2 = {'prec': [], 'rec': [], 'f1': []}
for rscore in rouge_scores:
    rouge1['prec'].append(rscore['rouge1'].precision)
    rouge1['rec'].append(rscore['rouge1'].recall)
    rouge1['f1'].append(rscore['rouge1'].fmeasure)
    
    rouge2['prec'].append(rscore['rouge2'].precision)
    rouge2['rec'].append(rscore['rouge2'].recall)
    rouge2['f1'].append(rscore['rouge2'].fmeasure)

In [69]:
rouge1_f = {key: statistics.mean(val) for key, val in rouge1.items()}
rouge2_f = {key: statistics.mean(val) for key, val in rouge2.items()}

# All Metrics 

In [71]:
print(f'Self-Bleu:            {sum(self_bleu)/len(self_bleu):.3f}')
print(f'Bert Score   (Prec):  {bprec:.3f}')
print(f'              (Rec):  {brec:.3f}')
print(f'               (F1):  {bf1:.3f}')
print(f'Rouge1 Score (Prec):  {rouge1_f["prec"]:.3f}')
print(f'              (Rec):  {rouge1_f["rec"]:.3f}')
print(f'               (F1):  {rouge1_f["f1"]:.3f}')
print(f'Rouge2 Score (Prec):  {rouge2_f["prec"]:.3f}')
print(f'              (Rec):  {rouge2_f["rec"]:.3f}')
print(f'               (F1):  {rouge2_f["f1"]:.3f}')

Self-Bleu:            0.253
Bert Score   (Prec):  0.474
              (Rec):  0.436
               (F1):  0.452
Rouge1 Score (Prec):  0.242
              (Rec):  0.127
               (F1):  0.144
Rouge2 Score (Prec):  0.023
              (Rec):  0.013
               (F1):  0.014
